# Phase advance between IPs in vdM scans

In the framework of the BB and vdM scan studies, we were requested to compute and distribute the phase advances of the optics used in stable beams. To improve the reproducibility of the data and limit black box approaches we prepared this simple notebook to get the requested tables.

In [ ]:
mySource='/eos/user/s/sterbini/MD_ANALYSIS/2019/VdMScans/PhaseAdvanceBetweenIPs.ipynb'
stack='96py3'
import cl2pd # do "pip install --user git+https://github.com/sterbini/cl2pd.git" to install
from cl2pd import importData
from cl2pd import dotdict
dotdict=dotdict.dotdict
pd=importData.pd     # is the pandas package
np=importData.np     # is the numpy package
cals=importData.cals # pytimber log class
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
%config InlineBackend.figure_format = 'retina' # retina display

In [20]:
from cpymad.madx import Madx
from matplotlib import pyplot as plt
myMad = Madx()


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.07.10 09:57:28      +
  ++++++++++++++++++++++++++++++++++++++++++++


# The MAD-X script
We use swan.cern.ch and the optics repositoris in EOS.

In [30]:
# Launching madx with the standard settings (from eos repositories)
myMad.input('''
! Sequence definition
option, -echo,-warn,-info;
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/const_for_aperture.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/lhc_as-built.seq";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/aperture_as-built.b1.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/aperture_as-built.b2.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/aper_tol_as-built.b1.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/aper_tol_as-built.b2.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/exp_pipe_model_after_LS1.madx";
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/aperture/exp_pipe_install_after_LS1.madx";
! Beam definition
beam, sequence=lhcb1, bv= 1,
  particle=proton, charge=1, mass=0.93827231,
  energy= 6500,   npart=0.85e11,kbunch=2748,
  ex=5.773983446153845e-10,ey=5.773983446153845e-10;
beam, sequence=lhcb2, bv=-1,
  particle=proton, charge=1, mass=0.93827231,
  energy= 6500,   npart=0.85e11,kbunch=2748,
  ex=5.773983446153845e-10,ey=5.773983446153845e-10;
! Strength definition
call, file="/eos/project/a/abpdata/lhc/optics/runII/2015/opt_19200_19000_19200_24000_coll.madx";
nrj=beam%lhcb1->pc/beam%lhcb1->charge;
''');

In [24]:
# twissing the lattice
myMad.input('''
use, sequence=lhcb1;
twiss, table=twiss_b1;
use, sequence=lhcb2;
twiss, table=twiss_b2;
''');
b1DF=myMad.table.twiss_b1.dframe()
b2DF=myMad.table.twiss_b2.dframe()

enter Twiss module
  
iteration:   1 error:   2.525108E-03 deltap:   0.000000E+00
orbit:  -3.056898E-05  6.269177E-08  4.857955E-06 -1.448609E-04  0.000000E+00  0.000000E+00
  
iteration:   2 error:   4.315597E-05 deltap:   0.000000E+00
orbit:   1.134273E-09 -7.053445E-10 -2.408938E-10 -1.450000E-04  0.000000E+00  0.000000E+00
  
iteration:   3 error:   4.337533E-09 deltap:   0.000000E+00
orbit:   2.590299E-11 -5.470202E-10  1.672655E-13 -1.450000E-04  0.000000E+00  0.000000E+00

++++++ table: summ

            length             orbit5               alfa            gammatr 
        26658.8832                 -0    0.0003202048804        55.88381246 

                q1                dq1            betxmax              dxmax 
        64.3099999        3.000000078         593.511501        2.720741306 

             dxrms             xcomax             xcorms                 q2 
       1.344104535      0.01545439897    0.0009885626082        59.32000003 

               dq2            

In [31]:
# retrieving the tunes
B1Q1=b1DF['mux'].values[-1]
B1Q2=b1DF['muy'].values[-1]
B2Q1=b2DF['mux'].values[-1]
B2Q2=b2DF['muy'].values[-1]
print(f'B1 H tune: {B1Q1}')
print(f'B1 V tune: {B1Q2}')
print(f'B2 H tune: {B2Q1}')
print(f'B2 V tune: {B2Q2}')
# filling a useful dictionary

# B1
B1=dotdict()
B1.atIP1=b1DF['name']=='ip1:1'
B1.atIP2=b1DF['name']=='ip2:1'
B1.atIP5=b1DF['name']=='ip5:1'
B1.atIP8=b1DF['name']=='ip8:1'
B1.atIP1l=b1DF['name']=='ip1.l1:1'


# B2
B2=dotdict()
B2.atIP1=b2DF['name']=='ip1:1'
B2.atIP2=b2DF['name']=='ip2:1'
B2.atIP5=b2DF['name']=='ip5:1'
B2.atIP8=b2DF['name']=='ip8:1'
B2.atIP1l=b2DF['name']=='ip1.l1:1'

B1 H tune: 64.30999990172486
B1 V tune: 59.32000002666296
B2 H tune: 64.30999990601576
B2 V tune: 59.32000002847266


# B1 summary

In [28]:
# NB: the orbit of the beams (namely IP separation and crossing angle could differ from the vdM fill settings)
b1DF[B1.atIP1 | B1.atIP2 | B1.atIP5 | B1.atIP8| B1.atIP1l][['name','s','betx','mux','bety','muy','x','px','y','py','dx','dpx']]

,name,s,betx,mux,bety,muy,x,px,y,py,dx,dpx
ip1,ip1:1,0.000000,19.111772,0.000000,19.263422,0.000000,2.590299e-11,-5.470202e-10,1.672655e-13,-1.450000e-04,-0.028090,-0.001066
ip2,ip2:1,3332.436584,18.862944,8.295954,18.929029,7.669167,-3.905104e-09,3.146726e-07,-3.385396e-13,1.953840e-04,-0.018981,0.001521
ip5,ip5:1,13329.289233,19.169696,31.975690,19.187829,29.648602,5.456222e-09,-1.039097e-10,-3.448094e-13,-3.200254e-15,-0.015047,-0.001544
ip8,ip8:1,23315.378984,24.114956,56.064831,23.903805,51.017069,9.757441e-09,-4.653714e-04,2.101193e-13,-1.948947e-06,-0.013657,-0.000533
ip1.l1,ip1.l1:1,26658.883200,19.111772,64.310000,19.263422,59.320000,2.590299e-11,-5.470202e-10,1.672627e-13,-1.450000e-04,-0.028090,-0.001066


# B2 summary

In [29]:
# NB: the orbit of the beams (namely IP separation and crossing angle could differ from the vdM fill settings)
b2DF[B2.atIP1 | B2.atIP2 | B2.atIP5 | B2.atIP8 | B2.atIP1l][['name','s','betx','mux','bety','muy','x','px','y','py','dx','dpx']]

,name,s,betx,mux,bety,muy,x,px,y,py,dx,dpx
ip1,ip1:1,0.000000,19.344438,0.000000,19.202845,0.000000,1.323483e-09,3.178942e-10,1.104578e-13,1.450000e-04,0.028278,-0.000782
ip2,ip2:1,3332.284216,18.856456,8.272802,18.964904,7.957720,1.431809e-09,-3.146798e-07,1.802691e-13,-1.953840e-04,-0.019935,-0.001338
ip5,ip5:1,13329.593967,19.097400,31.984398,19.200269,29.761319,9.546657e-10,1.033076e-11,6.666004e-13,-1.986384e-15,0.021519,-0.000772
ip8,ip8:1,23315.226616,24.347550,55.799011,23.946384,51.715754,8.990442e-10,4.653711e-04,7.248752e-13,1.948947e-06,-0.018350,0.000042
ip1.l1,ip1.l1:1,26658.883200,19.344438,64.310000,19.202845,59.320000,1.323483e-09,3.178942e-10,1.104609e-13,1.450000e-04,0.028278,-0.000782


# Phase advance between the IP with the "standard" optics configuration

In [13]:
auxB1=b1DF[B1.atIP1 | B1.atIP2 | B1.atIP5 | B1.atIP8| B1.atIP1l][['name','mux','muy']]
auxB2=b2DF[B2.atIP1 | B2.atIP2 | B2.atIP5 | B2.atIP8| B2.atIP1l][['name','mux','muy']]
myDict={'IPs':['IP1','IP2','IP5','IP8','IP1.L1'],'mux_B1': auxB1['mux'].values,'muy_B1': auxB1['muy'].values,
       'mux_B2': auxB2['mux'].values,'muy_B2': auxB2['muy'].values}
solution1=pd.DataFrame(myDict)
solution1

,IPs,mux_B1,muy_B1,mux_B2,muy_B2
0,IP1,0.000000,0.000000,0.000000,0.000000
1,IP2,8.295954,7.669167,8.272802,7.957720
2,IP5,31.975690,29.648602,31.984398,29.761319
3,IP8,56.064831,51.017069,55.799011,51.715754
4,IP1.L1,64.310000,59.320000,64.310000,59.320000


**NB:** the partial phase advances of B1 and B2 are different.

# Changing the experiments bumps
We repeat the analysis to find the phase advances between the IPs by changing the machine closed orbit.
The goal is to test the sensitivity to this parameter.

In [32]:
myMad.input('''
on_x1=0;
on_sep1= 0;
on_o1= 0;

on_x2=0;
on_x5=0;
on_sep5= 0;

on_x8=0;
on_alice=0;
on_lhcb=0;

on_o1= 0;
on_o5= 0;

use, sequence=lhcb1;
twiss, table=twiss_ipb1_flat;

use, sequence=lhcb2;
twiss, table=twiss_ipb2_flat;
''')
b1DF=myMad.table.twiss_ipb1_flat.dframe()
b2DF=myMad.table.twiss_ipb2_flat.dframe()

enter Twiss module
  
iteration:   1 error:   9.251000E-09 deltap:   0.000000E+00
orbit:  -7.075985E-11 -5.260818E-10  0.000000E+00  0.000000E+00  0.000000E+00  0.000000E+00

++++++ table: summ

            length             orbit5               alfa            gammatr 
        26658.8832                 -0    0.0003202008398        55.88416505 

                q1                dq1            betxmax              dxmax 
             64.31        3.005049683        593.5113395        2.822825194 

             dxrms             xcomax             xcorms                 q2 
       1.344676563    3.353872132e-08    9.423767048e-09        59.31999999 

               dq2            betymax              dymax              dyrms 
       2.997897113        609.8694967                 -0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            

In [33]:
# Result with the new machine configuration
auxB1=b1DF[B1.atIP1 | B1.atIP2 | B1.atIP5 | B1.atIP8| B1.atIP1l][['name','mux','muy']]
auxB2=b2DF[B2.atIP1 | B2.atIP2 | B2.atIP5 | B2.atIP8| B2.atIP1l][['name','mux','muy']]
myDict={'IPs':['IP1','IP2','IP5','IP8','IP1.L1'],'mux_B1': auxB1['mux'].values,'muy_B1': auxB1['muy'].values,
       'mux_B2': auxB2['mux'].values,'muy_B2': auxB2['muy'].values}
solution2=pd.DataFrame(myDict)
solution2

,IPs,mux_B1,muy_B1,mux_B2,muy_B2
0,IP1,0.000000,0.000000,0.000000,0.000000
1,IP2,8.295954,7.669167,8.272802,7.957720
2,IP5,31.975690,29.648602,31.984398,29.761319
3,IP8,56.064831,51.017069,55.799012,51.715754
4,IP1.L1,64.310000,59.320000,64.310000,59.320000


# Difference between the two configurations


In [34]:
solution1[['mux_B1','muy_B1','mux_B2','muy_B2']]-solution2[['mux_B1','muy_B1','mux_B2','muy_B2']]

,mux_B1,muy_B1,mux_B2,muy_B2
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,8.118508e-08,-2.396127e-08,-7.616989e-08,-6.599904e-09
2,4.398650e-10,-1.945075e-08,-6.726470e-09,9.467822e-09
3,-3.711982e-09,1.001098e-09,-8.038246e-08,2.203826e-08
4,-9.402230e-08,3.350141e-08,-9.383005e-08,3.279654e-08


**CONCLUSION:** as expected the two solutions are equivalent (error much smaller than what can be measured at the moment).